# Model 2: Toxicity Classification using KNN

Code to train a simple KNN model to predict toxicity

- use k-fold crossvalidation with grid-search to find best performing model on the training data
    - k==5
    - scoring metric = recall
- validate final model against a "hold-out" dataset
- fit final model using the full dataset and best hyper-parameters
- save trained model as pickle file (in case it is selected as the final model)

_**note on scoring metric:**_ choosing 'recall' as performance metric due to the potentail harm that can be caused be mis-labelling a toxic drug as harmless. In a real research scenario an in-silico model such as this would only be used to screen out obviously toxic drugs from a large list of candidates at an early stage and there must be more robust experimentally reliable tests performed downstream before the drug gets anywhere near a human being. 

---

## Imports and Constants



In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#string constants - centralise to avoid later 'finger trouble'
data_folder = "../data/"                     #relative folder path to the notebooks folder
data  = f"{data_folder}final-data.csv"       # in the real world datasets would be registered in a system like W&B to protect them from modification
model_folder = "../models/"

class_column = "Toxic"

---

## Data Prep

- Split dataset into input variables (X) and response variables (y)
- Create a hold-out set
    - use stratification to ensure proportion of toxic vs non-toxic is retained in the hold-out set

In [2]:
#read csv
toxicity_data = pd.read_csv(data)

#seperate in to X, y
X = toxicity_data.drop(class_column, axis=1)
y = toxicity_data[class_column]

#create a hold-out dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)


---

## Model training and hyper-parameter tuning


In [3]:

# Initialize the Random Forest classifier
knn_classifier = KNeighborsClassifier()

# Define the hyperparameters to search
param_grid = {
    'p': [2,3,4],
    'metric': ['minkowski'],
    'algorithm': ['auto', 'ball_tree', 'brute'],
    'leaf_size': [10,20,30,40]
}

# Create a k-fold cross-validation splitter
k_fold = KFold(n_splits=5, shuffle=True, random_state=666)

# Perform hyperparameter search with k-fold cross-validation
grid_search = GridSearchCV(knn_classifier, param_grid, cv=k_fold, scoring="accuracy")
grid_search.fit(X_train, y_train)



c:\Users\ktqc696\OneDrive - AZCollaboration\IC-ML\Portfolio\.venv\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=KFold(n_splits=5, random_state=666, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'brute'],
                         'leaf_size': [10, 20, 30, 40], 'metric': ['minkowski'],
                         'p': [2, 3, 4]},
             scoring='accuracy')

In [4]:
# Display the best hyperparameters and the corresponding mean cross-validated score
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'p': 4}


---

## Final model evaluation

- use best hyper-parameters
- fit final model to full dataset
    - evaluate model on hold out set
    - compare to full dataset result
- save trained model to disc for later use

In [5]:
# Fit model to full dataset and evaluate against held out data

#define classifier
final_knn_classifier = KNeighborsClassifier(metric    =grid_search.best_params_["metric"], 
                                           p         =grid_search.best_params_["p"], 
                                           algorithm =grid_search.best_params_["algorithm"], 
                                           leaf_size =grid_search.best_params_["leaf_size"])
#fit model with best hyper-parameters
final_knn_classifier.fit(X,y)

# Hold Out data ---------------------------
#make predictions
y_test_pred = final_knn_classifier.predict(X_test)

#compare with actuals
original_toxic = np.count_nonzero(y_test)
correctly_predicted_toxic = np.count_nonzero(y_test & y_test_pred)

recall = correctly_predicted_toxic/original_toxic
accuracy = np.count_nonzero(y_test==y_test_pred) / len(y_test_pred)

print("'Hold out' test set results")
print(f"\tRecall:\t\t{recall:.2f}")
print(f"\tAccuracy:\t{accuracy:.2f}")

# Full data set ------------------------
#make predictions
y_pred = final_knn_classifier.predict(X)

#compare with actuals
original_toxic = np.count_nonzero(y)
correctly_predicted_toxic = np.count_nonzero(y & y_pred)

recall = correctly_predicted_toxic/original_toxic
accuracy = np.count_nonzero(y==y_pred) / len(y_pred)

print("Full dataset results")
print(f"\tRecall:\t\t{recall:.2f}")
print(f"\tAccuracy:\t{accuracy:.2f}")

with open(f'{model_folder}model2-knn-trained.pkl', 'wb') as f:
        pickle.dump(final_knn_classifier, f)

'Hold out' test set results
	Recall:		0.73
	Accuracy:	0.89
Full dataset results
	Recall:		0.55
	Accuracy:	0.77
